In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
%cd Z:\PPMI_Data\Excels\Only_Female\GeneCpG\Gastro
data = pd.read_csv('Gastro_with_MethylomeDetails_Female.csv')
data.head(1)

Z:\PPMI_Data\Excels\Only_Female\GeneCpG\Gastro


,Sentrix,PATNO,Gender,APPRDX,HYS,Age,AgeCate,SCOPA_Gastro_Category,cg03441945,cg21094410,...,cg05766881,cg01708543,cg10933281,cg04735449,cg15856662,cg11864127,cg14420798,cg08859349,cg01481205,cg19228755
0,200973410159_R03C01,3000,2,2,0,69.1,4,0,0.89241,0.015756,...,0.874283,0.88697,0.903544,0.918856,0.869453,0.889527,0.853304,0.862654,0.929857,0.889773


In [3]:
X = data.drop(['Sentrix','PATNO','HYS','Gender','Age','AgeCate','SCOPA_Gastro_Category'], axis = 1)
y = data['APPRDX']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   APPRDX  cg03441945  cg21094410  cg22926093  cg25025609  cg16704802  \
0       2    0.892410    0.015756    0.011101    0.018491    0.020476   
1       1    0.922532    0.013309    0.012393    0.018230    0.018660   

   cg06228737  cg00220102  cg19016494  cg12247200  ...  cg05766881  \
0    0.962482    0.677222    0.029560    0.929821  ...    0.874283   
1    0.924050    0.577240    0.019605    0.937446  ...    0.843815   

   cg01708543  cg10933281  cg04735449  cg15856662  cg11864127  cg14420798  \
0    0.886970    0.903544    0.918856    0.869453    0.889527    0.853304   
1    0.861038    0.870442    0.896617    0.857573    0.832123    0.807071   

   cg08859349  cg01481205  cg19228755  
0    0.862654    0.929857    0.889773  
1    0.846446    0.914551    0.874165  

[2 rows x 36556 columns]


In [4]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [5]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.8

In [6]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=2)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100


BorutaPy finished running.

Iteration: 	10 / 100
Confirmed: 	1
Tentative: 	0
Rejected: 	36555


BorutaPy(estimator=RandomForestClassifier(n_estimators=14,
                                          random_state=RandomState(MT19937) at 0x13FB8C06B40),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x13FB8C06B40, verbose=1)

In [7]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  1


In [8]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [9]:
Top50 = selected_rf_features.iloc[:51, :]
Top50.head(3)

,CpG,Ranking
0,APPRDX,1
14282,cg06407053,2
12003,cg18248290,3


In [10]:
Top50.to_csv ('Gastro_BorutaFemale_Top50.csv', index = False)
selected_rf_features.to_csv ('Gastro_BorutaFemale_All.csv', index = False)

#Identify a string in dataframe
substring = 'cg15328505'
selected_rf_features[selected_rf_features.apply(lambda row: row.astype(str).str.contains(substring, case=False).any(), axis=1)]
